
**Problem**
-------

Create a Culinary Assistant agent that:
> 1) Finds recipes using a given main ingredient.

> 2) Filters them by dietary restrictions (e.g., excludes shrimp).

> 3) Provides clear, step-by-step cooking instructions for a selected recipe.

Notes:

- Disables telemetry to avoid OpenTelemetry / network timeouts.
- Uses CrewAI’s Agent/Task/Crew orchestration.


In [ ]:
! pip install -q crewai==0.80.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.3/242.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 63.5 MB

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

In [ ]:
# --- Configuration ---
os.environ["OPENAI_API_KEY"] = "replace this with AIR key (removing my personal one)"

In [ ]:
# Disable CrewAI telemetry to avoid network timeouts
os.environ["CREWAI_TELEMETRY_ENABLED"] = "false"

In [ ]:
class CulinaryCrew:
    """A class to manage the Culinary Assistant crew."""

    def __init__(self, main_ingredient: str, dietary_restrictions: list[str]):
        """Initializes the crew with user-defined inputs."""
        self.main_ingredient = main_ingredient
        self.dietary_restrictions = dietary_restrictions
        # Use a flexible LLM, defaulting to gpt-4
        self.llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME", "gpt-4.1-nano"))

    def setup_crew(self):
        """Defines the agent, tasks, and crew for the culinary assistant."""

        # --- Agent Definition ---
        culinary_assistant = Agent(
            llm=self.llm,
            role="Culinary Assistant",
            goal=(
                "Find recipes based on a main ingredient, filter them by dietary "
                "restrictions, and provide clear, step-by-step cooking instructions."
            ),
            backstory=(
                "You are an expert culinary assistant, renowned for your ability to find "
                "the perfect recipe. You tailor your suggestions to specific ingredients "
                "and dietary needs, and then guide users through the cooking process "
                "with simple, easy-to-follow steps."
            ),
            verbose=True,
        )

        # --- Task Definitions ---
        find_and_filter_recipes = Task(
            description=(
                "Find a single, suitable recipe featuring '{main_ingredient}'. "
                "Crucially, the recipe must NOT contain any of the following "
                "restricted ingredients: {dietary_restrictions}. "
                "Present the chosen recipe's title, a brief rationale for why it was "
                "chosen, a summary of its steps, cooking time, and serving size."
            ),
            expected_output=(
                "A single recipe object including: title, rationale, ingredients, "
                "a summary of steps, cooking time, and serving size. The recipe must "
                "adhere strictly to the dietary restrictions."
            ),
            agent=culinary_assistant,
        )

        guide_recipe_steps = Task(
            description=(
                "Using the recipe provided from the previous task, create clear, "
                "numbered, step-by-step cooking instructions. The instructions should "
                "be easy for a novice cook to follow. Include details on preparation, "
                "cooking times, temperatures, and serving suggestions."
            ),
            expected_output=(
                "A well-formatted, numbered list of cooking instructions from "
                "start to finish, including prep and serving tips."
            ),
            # The 'context' parameter ensures this task uses the output of the first task
            context=[find_and_filter_recipes],
            agent=culinary_assistant,
        )

        # --- Crew Definition ---
        return Crew(
            agents=[culinary_assistant],
            tasks=[find_and_filter_recipes, guide_recipe_steps],
            process=Process.sequential,  # Tasks will be executed one after another
            verbose=True,
        )

    def run(self):
        """Kicks off the crew to perform its tasks."""
        crew = self.setup_crew()
        result = crew.kickoff(
            inputs={
                "main_ingredient": self.main_ingredient,
                "dietary_restrictions": self.dietary_restrictions,
            }
        )
        return result

In [ ]:
# --- Main Execution ---
if __name__ == "__main__":
    print("## Welcome to the Culinary Assistant Crew! ##")
    print("-------------------------------------------")

    # Define your inputs here
    main_ingredient = "tomato"
    dietary_restrictions = ["shrimp", "dairy"]

    # Initialize and run the crew
    culinary_crew = CulinaryCrew(main_ingredient, dietary_restrictions)
    final_result = culinary_crew.run()

    print("\n\n########################")
    print("## Here is the final result:")
    print("########################\n")
    print(final_result)

## Welcome to the Culinary Assistant Crew! ##
-------------------------------------------
# Agent: Culinary Assistant
## Task: Find a single, suitable recipe featuring 'tomato'. Crucially, the recipe must NOT contain any of the following restricted ingredients: ['shrimp', 'dairy']. Present the chosen recipe's title, a brief rationale for why it was chosen, a summary of its steps, cooking time, and serving size.


# Agent: Culinary Assistant
## Final Answer: 
{
  title: "Vegan Tomato Basil Soup",
  rationale: "This recipe was chosen because it highlights fresh tomatoes as the main ingredient while being completely dairy-free and free of shrimp or any seafood. It’s a hearty, flavorful, and healthy option suitable for vegans and those avoiding dairy, making it a perfect choice given the restrictions.",
  ingredients: [
    "2 tablespoons olive oil",
    "1 large onion, chopped",
    "4 cloves garlic, minced",
    "6 large ripe tomatoes, chopped",
    "2 cups vegetable broth",
    "1 teasp